# Code 


In [202]:
#### Variables Globales 

# Données initiales
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg

# Variables de simulation
PFCmin = 2500 # en kg
PFCmax = 5000 # en kg
Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %
poche_i = 1250 # en kg
tmin = 0 # en minutes
tmax = 4*60 # en minutes
temps_serie = 3  # en min
temps_traitement = 10 # en min
numPoche = 3
numFours = 1

# Dictionnaires pour stocker les variables et états
cles = ["chargement_et_fusion", "Four 1", "Four 2", "Four 3", "Four 4", "Four 5"]
times_dict = {cle: [] for cle in cles}
PFCvalues_dict = {cle: [] for cle in cles}
Mgvalues_dict = {cle: [] for cle in cles}




PFC = 3750  # Initialisation de la fonte consommée
Mg =  Mgmax  # Initialisation du pourcentage de Mg
t = 0 # en minutes
etat = "etat_Consom"  # État initial : production de moules

temps_serie_courant = temps_serie



# ------------------------------------------------


def calcul_duree_limite_et_delai_traitement(Mg, PFC):
    """
    Cette fonction calcule la durée limite avant l'épuisement de Mg ou de PFC et le délai avant de lancer la prochaine poche de traitement.

    Args:
    Mg (float): Quantité initiale de Mg.
    PFC (float): Quantité initiale de PFC.

    Returns:
    int: DurreLimite, durée limite avant l'épuisement de Mg ou PFC.
    int: delai_avt_traitement, délai avant de lancer la prochaine poche de traitement.
    """

    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_serie, temps_traitement
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire

    # Copie des listes pour éviter les modifications
    masses_grappes = liste_masses_grappes_moules.copy()
    cadences_moule = liste_cadences_moule_par_heure.copy()
    quantites_moules = liste_quantites_moules_a_produire.copy()

    ## on calcule le temps mis avant l'épuisement du MgDispo
    # On calcule le pourcentage de Mg consommable
    MgDispo = Mg - Mgmin
    # On calcule le temps mis avant l'épuisement du MgDispo
    DurreMg = MgDispo/eC

    ## on calcule le temps avant l'épuisement de la fonte (DurrePFC)
    # On calcule a fonte disponible pour la production de Moules
    PFCDispo = PFC - PFCmin
    PFCrestant = PFCDispo

    # Avec le PFCDispo, on veux calculer la durée avant l'épuisement 
    # de la fonte disponible et aussi calcule masse de la fonte consommer
    DurrePFC = 0
    PFCrestant = PFCDispo
    # Tant que l'on a de la fonte disponible
    # while PFCrestant != 0 :
    # while True:
    while cadences_moule :
        masse_grappe_i = masses_grappes[0]
        cadence_moule_par_heure_i = cadences_moule[0]
        quantite_moules_a_produire_i = quantites_moules[0]
        # La cadence de consommation en kg/min du i-ème modèle
        cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
        # Quantités de moules du i-ème modèle produits en une minute
        cadence_moule_i = int(cadence_fonte_i / masse_grappe_i) # en unités/min
        # on calcule le nombre de moules du i-ème modèle que l'on pourrait faire 
        # avec cette onte disponible
        nbMoules_i = int(PFCrestant/masse_grappe_i)


        # Si la fonte disponible permet de produire tous les moules du i-èmes modèles
        if nbMoules_i >=  quantite_moules_a_produire_i :
            # Alors on met à jour la fonte total consommer et la durée DurrePFC
            #  après avoir produit tous les moules du i-ème modèles
            PFCrestant -= quantite_moules_a_produire_i* masse_grappe_i # en kg
            DurrePFC += quantite_moules_a_produire_i /cadence_moule_i # en min

            # On change le modèle i par le modèle i+1
            # Pendant ce changement, on atttend temps_Serie minutes
            DurrePFC += temps_serie

            # On passe au modèle i+1, donc on met à jour le programme des moules
            masses_grappes.pop(0)
            cadences_moule.pop(0)
            quantites_moules.pop(0)

        # Si la fonte disponible ne permet pas de produire tous les moules du i-èmes modèles
        else :
            # Alors on met à jour la fonte total consommer et la durée DurrePFC
            # après avoir produit les moules du i-ème modèles que l'on peut faire
            
            PFCrestant -= nbMoules_i* masse_grappe_i # en kg
            DurrePFC += nbMoules_i /cadence_moule_i # en min


            # On met à jour le programme de productions des moules
            quantites_moules[0] -= nbMoules_i

            # Meme si le PFCrestant n'est pas exactement nul, 
            # on arrete la production de moules
            break

    
    # On obient finalement la deadline (le temps dont on dispose)
    # avant que Mg ou PFC devienne insufisant
    DurreLimite = int(min(DurreMg,DurrePFC))  # en min

    # On a aussi le delai dont on dispose avant de 
    # Lancer la prochaine poche de traitement
    delai_avt_traitement = (DurreLimite - temps_traitement)

    return DurreLimite, delai_avt_traitement


def ProduirMoulesParMinute(Mg, PFC, quantite_moules_a_produire_i, cadence_fonte_i, cadence_moule_i):
    
    # Constantes dans ce programme
    global Mgmin, eC, PFCmin, temps_serie, temps_traitement
    # Variables dans ce programme
    global liste_masses_grappes_moules, liste_cadences_moule_par_heure, liste_quantites_moules_a_produire

    # Mise à jour du pourcentage de Mg et du poids fonte coulée
    PFC -= cadence_fonte_i
    Mg -= eC

    if PFC > PFCmax:
        print("On a dépassé la capacité du Four de Coulée")
    

    quantite_moules_restants = quantite_moules_a_produire_i - cadence_moule_i

    if debut < t < fin:
        print("Ici2,t =",t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i, quantite_moules_restants)

    # On a fini de réaliser le i-ème modèle alors on passe au i+1-ème modèle
    if quantite_moules_restants <= 0:
        etat = "etat_Serie"
        # Suppression des éléments déjà traités dans les listes
        liste_masses_grappes_moules.pop(0)
        liste_quantites_moules_a_produire.pop(0)
        liste_cadences_moule_par_heure.pop(0)

        # Si on a fini de tout produire alors on stoppe la procédure 
        # On passe à l'état Fin
        if not liste_masses_grappes_moules:
            etat = "etat_Fin"

        if debut < t < fin:
            print("Ici3,t =",t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i, quantite_moules_restants)

    # On a fini de réaliser le j-ème moule du i-ème modèle alors on passe au j-ème moule
    # En restant dans l'état consommation 
    else:
        if debut < t < fin:
            print("Ici3,t =",t, Mg, PFC, cadence_fonte_i, quantite_moules_a_produire_i, quantite_moules_restants)

        liste_quantites_moules_a_produire[0] = quantite_moules_restants
        etat = "etat_Consom"
    
    return Mg, PFC, etat




In [207]:
# Données initiales
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg
PFC = 3750  # Initialisation de la fonte consommée
Mg =  Mgmax  # Initialisation du pourcentage de Mg
t = 0 # en minutes
etat = "etat_Consom"  # État initial : production de moules

temps_serie_courant = temps_serie
numPoche = 3
numFours = 1

DurreLimite, delai_avt_traitement = calcul_duree_limite_et_delai_traitement(Mg, PFC)

# Boucle de simulation
while t < tmax :

    # Les resultats des 2 boutons!!!
    Fin_Panne = False
    Panne = False
    # Si l'utilisateur clique sur le bouton "Debut Panne"
    ## Alors on est dans etat == "etat_Panne" 
    if Panne :
        etat == "etat_Panne"


    # Si l'utilisateur clique sur le bouton "Fin Panne"
    ## Alors on est dans etat == "etat_Consom" 
    if Fin_Panne :
        etat = "etat_Consom"


    # Gestion de la poche de traitement
    if DurreLimite <= 0:
        PFC += poche_i
        Mg = Mgmax
        DurreLimite, delai_avt_traitement = calcul_duree_limite_et_delai_traitement(Mg, PFC)
        print("Poche ajoutée, recalcul des délais:", DurreLimite, delai_avt_traitement)
    else:
        if delai_avt_traitement <= 0:
            print("Lancer le traitement !!", t)
        else:
            print("Avant Lancement de la PT, il reste", delai_avt_traitement, t)
            delai_avt_traitement -= 1

        print("Avant L'ajout de la PT, il reste", DurreLimite, t)
        DurreLimite -= 1
    



    if etat == "etat_Consom":
        masse_grappe_i =liste_masses_grappes_moules[0]
        cadence_moule_par_heure_i = liste_cadences_moule_par_heure[0]
        quantite_moules_a_produire_i = liste_quantites_moules_a_produire[0]
        # La cadence de consommation en kg/min du i-ème modèle
        cadence_fonte_i = cadence_moule_par_heure_i / 60 * masse_grappe_i  # en kg/min
        # Quantités de moules du i-ème modèle produits en une minute
        cadence_moule_i = int(cadence_fonte_i / masse_grappe_i) # en unités/min


        if  debut < t and  t < fin :
            print("Ici1", Mg, PFC, cadence_fonte_i,quantite_moules_a_produire_i)
        
        # Si l'on peut produire des moules (c-a-d Mg et PFC sont okay) alors on on le fait !!
        if (PFC - cadence_fonte_i >= PFCmin) and (Mg - eC >= Mgmin):
            Mg, PFC, etat = ProduirMoulesParMinute(Mg, PFC, quantite_moules_a_produire_i, cadence_fonte_i, cadence_moule_i)
                
        else:
            print("on peut plus produire des moules (car Mg et PFC)",t)     
            if PFC > PFCmax :
                print("On a depasser pas la capacité du Fours de Coulée")     

    elif etat == "etat_Serie":
        # print("Changement de Serie")
        Mg -= eC
        temps_serie_courant -= 1

        if temps_serie_courant <= 0:
            if  debut < t and  t < fin :
                print("Fin Temps Serie",Mg, PFC, cadence_fonte_i)
            
            temps_serie_courant = temps_serie
            etat = "etat_Consom"
        else:
            etat == "etat_Serie" 

    elif etat == "etat_Panne":
        Mg -= eC
        etat == "etat_Panne"
        if  debut < t and  t < fin :
            print("Fin Panne",Mg, PFC, cadence_fonte_i)

    elif etat == "etat_Fin":
        break
    



    

    # Mise à jour du temps
    t += 1
    cle = "Four 1"
    cle = f"Four {numFours}" 
    PFCvalues_dict[cle].append(PFC)
    Mgvalues_dict[cle].append(Mg)
    times_dict[cle].append(t)


Avant L'ajout de la PT, il reste 10 0
t =  0
Ici1 0.045 3750 130.46666666666667 121
Ici2,t = 0 0.0445 3619.5333333333333 130.46666666666667 121 118
Ici3,t = 0 0.0445 3619.5333333333333 130.46666666666667 121 118
Avant L'ajout de la PT, il reste 9 1
t =  1
Ici1 0.0445 3619.5333333333333 130.46666666666667 118
Ici2,t = 1 0.044 3489.0666666666666 130.46666666666667 118 115
Ici3,t = 1 0.044 3489.0666666666666 130.46666666666667 118 115
Avant L'ajout de la PT, il reste 8 2
t =  2
Ici1 0.044 3489.0666666666666 130.46666666666667 115
Ici2,t = 2 0.0435 3358.6 130.46666666666667 115 112
Ici3,t = 2 0.0435 3358.6 130.46666666666667 115 112
Avant L'ajout de la PT, il reste 7 3
t =  3
Ici1 0.0435 3358.6 130.46666666666667 112
Ici2,t = 3 0.043 3228.133333333333 130.46666666666667 112 109
Ici3,t = 3 0.043 3228.133333333333 130.46666666666667 112 109
Avant L'ajout de la PT, il reste 6 4
t =  4
Ici1 0.043 3228.133333333333 130.46666666666667 109
Ici2,t = 4 0.042499999999999996 3097.6666666666665 130.46

In [ ]:
    # Si c'est le moment de lancer le traitement de la poche 
    # ( delai_avt_traitement_courant <= 0 )
    ## Alors dire à la fusion de lancer le traitement, 
    ## Puis dans DurreLimite_courant minutes, on ajoute la poche 
    ## On ajoute la poche dans l'etat consom, Serie uniquement
    ## Peut-etre à la fin de la Panne
    # print("t =",t,DurreLimite_courant, delai_avt_traitement_courant)


    # Si c'est le moment d'ajouter la Poche de Traitement
    if DurreLimite <= 0 :
        # alors on l'ajoute
        PFC += poche_i
        Mg = Mgmax
    
        # On calcule le delai du prochain Traitement 
        DurreLimite, delai_avt_traitement = calcul_duree_limite_et_delai_traitement(Mg, PFC)
          
    # Si c'est pas encrore le moment d'ajouter la Poche de Traitement
    else :
        # alors on dit quand ca serait le moment d'ajouter la Poche de Traitement
     
        # Si c'est le moment d'annoncer le lancer la Poche de Traitement
        if delai_avt_traitement <= 0 :
            pass
            # alors on le dit 
            # print("Lancer le traitement !!",t)

        # Si c'est pas encrore le moment d'annoncer le lancer la Poche de Traitement
        else :
            # alors on dit quand ca serait le moment d'annoncer le lancer la Poche de Traitement
            print("Avant Lancement de la PT, il reste", delai_avt_traitement,t) 

            # On mets à jour le delai vu qu'une minute est écoulé
            delai_avt_traitement -= 1

        # alors on dit quand ca serait le moment d'ajouter la Poche de Traitement
        print("Avant L'ajout de la PT, il reste", DurreLimite,t) 

        # On mets à jour le delai vu qu'une minute est écoulé
        DurreLimite -= 1



Avant L'ajout de la PT, il reste 9 avec t = 0
Ici1,t = 0 0.045 3750 130.46666666666667 121
Ici2,t = 0 0.0445 3619.5333333333333 130.46666666666667 121 118
Ici3,t = 0 0.0445 3619.5333333333333 130.46666666666667 121 118
Avant L'ajout de la PT, il reste 8 avec t = 1
Ici1,t = 1 0.0445 3619.5333333333333 130.46666666666667 118
Ici2,t = 1 0.044 3489.0666666666666 130.46666666666667 118 115
Ici3,t = 1 0.044 3489.0666666666666 130.46666666666667 118 115
Avant L'ajout de la PT, il reste 7 avec t = 2
Ici1,t = 2 0.044 3489.0666666666666 130.46666666666667 115
Ici2,t = 2 0.0435 3358.6 130.46666666666667 115 112
Ici3,t = 2 0.0435 3358.6 130.46666666666667 115 112
Avant L'ajout de la PT, il reste 6 avec t = 3
Ici1,t = 3 0.0435 3358.6 130.46666666666667 112
Ici2,t = 3 0.043 3228.133333333333 130.46666666666667 112 109
Ici3,t = 3 0.043 3228.133333333333 130.46666666666667 112 109
Avant L'ajout de la PT, il reste 5 avec t = 4
Ici1,t = 4 0.043 3228.133333333333 130.46666666666667 109
Ici2,t = 4 0.04249

# Courbe interactive

In [ ]:
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import asyncio
import time

def initialize_variables(initial_time_step, initial_t):
    """Initialise les variables de contrôle avec des paramètres."""
    global running, current_function, time_step, xdata, ydata, t
    running = False
    current_function = 0
    time_step = initial_time_step
    xdata = []
    ydata = []
    t = initial_t
def create_figure():
    """Crée et configure la figure Plotly."""
    fig = go.FigureWidget()
    scatter = fig.add_scatter(mode='lines+markers').data[0]
    fig.update_layout(title="Courbe Interactive", xaxis_title="Temps", yaxis_title="Valeur")
    return fig, scatter

async def update_data():
    """Fonction de mise à jour des données."""
    global running, current_function, time_step, xdata, ydata, t
    while True:
        if running:
            
            xdata.append(t)
            if current_function == 0:
                ydata.append(np.sin(t))
            elif current_function == 1:
                ydata.append(2)

            with fig.batch_update():
                scatter.update(x=xdata, y=ydata)
                
            t += time_step
            time.sleep(0.5)
        
        await asyncio.sleep(time_step)

def start_or_next_function(button):
    """Fonction pour démarrer ou changer la fonction."""
    global running, current_function
    current_function = (current_function + 1) % 2
    running = True

def stop_plot(button):
    """Fonction pour arrêter la mise à jour."""
    global running
    running = False

def reset_plot(button):
    """Fonction pour réinitialiser les données et le temps t."""
    global running, xdata, ydata, t
    running = False
    xdata = []
    ydata = []
    t = 0
    with fig.batch_update():
        scatter.update(x=xdata, y=ydata)

def create_buttons():
    """Crée les boutons interactifs."""
    start_button = widgets.Button(description="Start/Next Function")
    stop_button = widgets.Button(description="Stop")
    reset_button = widgets.Button(description="Reset")

    start_button.on_click(start_or_next_function)
    stop_button.on_click(stop_plot)
    reset_button.on_click(reset_plot)

    display(widgets.HBox([start_button, stop_button, reset_button]), fig)

def main(initial_time_step=1, initial_t=0):
    """Fonction principale pour démarrer l'application."""
    initialize_variables(initial_time_step, initial_t)
    global fig, scatter
    fig, scatter = create_figure()
    create_buttons()
    loop = asyncio.get_event_loop()
    loop.create_task(update_data())

# Appel de la fonction principale pour démarrer l'application
main()


# Simulation de la Production de Moules

On a 4000 kg de fontes dans le four de Coulée avec un pourcentage de Mg = 0.0045 %.
On consonmme 36 Kg de fonte par minute dans le four de Coulée.
On perd 0.005 % de Mg par minute.

Tu es un mathématicien et tu dois Tracer la courbe d'évolution du Mg
dans le four de Coulée. 

Si tu as des questions ou s'il te manque certaines informations, n'hésite pas à me les demander !!

Soit PFC_Consom, la fonte utiliser pour produire les moules 
Soit Mg, le pourcentage de Mg une  variable qui intervient lors de la production des moules
Que l'on conservera dans un dictionnaire : times_dict, PFC_Consomvalues_dict, Mgvalues_dict

Soit temps_serie, le temps mis pour passer au modeles suivant

Soit PFCmin et Mgmin, la masse de fonte minimal et le pourcentage de Mg minimal
que peuvent prendre PFC_Consom et Mg


A chaque temps t, on a 4 états possibles:
 - Soit on produit des moules 
 - Soit on change de modeles
 - Soit on est en Panne
 - Soit on est à la Fin

Lors que l'on produit des moules :

    Si etat est egal à etat_Consom alors # On verifie que l'on est en etat consommation dans le temps

        si PFC_Consom - cadence_consommation_kg_min et Mg - 0.0005  >= PFCmin et Mgmin alors  # Si l'on peut consommer alors on le fait !!
            
            PFC_Consom -= cadence_consommation_kg_min et Mg -= 0.0005

            # Quantité de moules du i-ème modèle produits en une minute
            cadence_moule_par_minute = cadence_consommation_kg_min / masse_grappe_moule  # en unités/min

            # Quantité de moules restants du i-ème modèle après une minute
            quantite_moules_restants = quantite_moules_a_produire - cadence_moule_par_minute # en unités

            # on met à jour liste_quantites_moules_a_produire et liste_masses_grappes_moules
            si quantite_moules_restants est nul alors 
                - Le temps passe a l'etat changement de Modeles
                - on passe au modele suivant dans les listes : 
                    # Suppression des éléments déjà traités dans les listes
                    liste_masses_grappes_moules.pop(0)
                    liste_quantites_moules_a_produire.pop(0)
                    liste_cadences_moule_par_heure.pop(0)
            sinon 
                - remplacer quantite_moules_a_produire par quantite_moules_restants
                - Le temps reste a l'etat on produit des moules

            si liste_masses_grappes_moules est vide alors 
                - Le temps passe a l'etat Fin ( on arrete toute la proccedure )
                

        Sinon # Si l'on ne peut pas consommer alors on rajoute de la fonte venant de la poche et on consomme
            PFC_Consom += 1250 et Mg = 0.0045 # on rajoute de la fonte venant de la poche 

            # Maintenant on peut consommer !!
            PFC_Consom -= cadence_consommation_kg_min et Mg -= 0.0005

            # Quantité de moules du i-ème modèle produits en une minute
            cadence_moule_par_minute = cadence_consommation_kg_min / masse_grappe_moule  # en unités/min

            # Quantité de moules restants du i-ème modèle après une minute
            quantite_moules_restants = quantite_moules_a_produire - cadence_moule_par_minute # en unités

            # on met à jour liste_quantites_moules_a_produire et liste_masses_grappes_moules
            si quantite_moules_restants est nul alors 
                - Le temps passe a l'etat changement de Modeles
                - on passe au modele suivant dans les listes : 
                    # Suppression des éléments déjà traités dans les listes
                    liste_masses_grappes_moules.pop(0)
                    liste_quantites_moules_a_produire.pop(0)
                    liste_cadences_moule_par_heure.pop(0)
            sinon 
                - remplacer quantite_moules_a_produire par quantite_moules_restants
                - Le temps reste a l'etat on produit des moules

            si liste_masses_grappes_moules est vide alors 
                - Le temps passe a l'etat Fin ( on arrete toute la proccedure )



Lors que l'on change de modeles :

    Si etat est egal à etat_Serie alors # On verifie que l'on est en etat Changement de Serie dans le temps
        Mg -= 0.0005 # Mg perd son pourcentage
        temps_serie -= 1 # On decremente temps_serie de 1   

        Si FC_Consom - cadence_consommation_kg_min et Mg - 0.0005  <= PFCmin et Mgmin alors  # Si l'on ne peut pas consommer alors on ajoute de la fonte !!
            PFC_Consom += 1250 et Mg = 0.0045 # on rajoute de la fonte venant de la poche 
    
        Si PFC_Consom >= PFCMax :
            "Error on est bloqué"

        Si temps_serie est nul alors 
            - Le temps passe à l'état produire des moules
        Sinon 
            - Le temps reste à temps_serie 



Lors que l'on est en Panne :

    Si etat est egal à etat_Panne alors # On verifie que l'on est en etat panne dans le temps
        Mg -= 0.0005 #Mg perd son pourcentage
        temps_Panne -= 1 # On decremente temps_Panne de 1   
        Si temps_Panne est nul alors 
            - Le temps passe à l'état produire des moules
        Sinon 
            - Le temps reste à temps_panne 



Remarques : 
    L'utilisateur doit avoir un bouton pour pouvoir activer l'etat en Panne

In [93]:
# Données initiales
liste_cadences_moule_par_heure = [190, 190, 190, 210, 190, 190, 190, 190, 190]  # en unités/heure
liste_quantites_moules_a_produire = [121, 50, 35, 10, 100, 50, 35, 50, 40]  # en unités
liste_masses_grappes_moules = [41.2, 39.36, 15.6, 19.94, 41.20, 39.36, 15.60, 41.20, 29.65]  # en kg

# Variables de simulation
PFCmin = 2500 # en kg
PFCmax = 5000 # en kg
Mgmin = 0.035 # en %
Mgmax = 0.045 # en %
eC = 0.0005 # en %
poche_i = 1250 # en kg
tmin = 0 # en minutes
t = 0 # en minutes


temps_serie = 10  # Initialisation du temps pour changer de modèle
temps_Panne = 10  # Initialisation du temps pour la panne

# Dictionnaires pour stocker les variables et états
cles = ["chargement_et_fusion", "Four 1", "Four 2", "Four 3", "Four 4", "Four 5"]
times_dict = {cle: [] for cle in cles}
PFC_Consomvalues_dict = {cle: [] for cle in cles}
Mgvalues_dict = {cle: [] for cle in cles}


PFC_Consom = 3750  # Initialisation de la fonte consommée
Mg =  Mgmax  # Initialisation du pourcentage de Mg
tmax = 4*60 # en minutes

# Initialisation des états
etat = "etat_Consom"  # État initial : production de moules

temps_serie_courant = temps_serie
temps_Panne_courant = temps_Panne
numPoche = 3
numFours = 1



#  if  debut < t and  t > fin :
debut,fin = 73,73


def ProduirMoulesParMinute(Mg, PFC_Consom, quantite_moules_a_produire, cadence_consommation_kg_min, cadence_moule_par_minute):
    
    # Mise à jour du pourcentage de Mg et du poids fonte coulée
    PFC_Consom -= cadence_consommation_kg_min
    Mg -= eC

    if PFC_Consom > PFCmax:
        print("On a dépassé la capacité du Four de Coulée")
    

    quantite_moules_restants = quantite_moules_a_produire - cadence_moule_par_minute

    if debut < t < fin:
        print("Ici2", Mg, PFC_Consom, cadence_consommation_kg_min, quantite_moules_a_produire, quantite_moules_restants)

    # On a fini de réaliser le i-ème modèle alors on passe au i+1-ème modèle
    if quantite_moules_restants <= 0:
        etat = "etat_Serie"
        # Suppression des éléments déjà traités dans les listes
        liste_masses_grappes_moules.pop(0)
        liste_quantites_moules_a_produire.pop(0)
        liste_cadences_moule_par_heure.pop(0)

        # Si on a fini de tout produire alors on stoppe la procédure 
        # On passe à l'état Fin
        if not liste_masses_grappes_moules:
            etat = "etat_Fin"

        if debut < t < fin:
            print("Ici3", Mg, PFC_Consom, cadence_consommation_kg_min, quantite_moules_a_produire, quantite_moules_restants)

    # On a fini de réaliser le j-ème moule du i-ème modèle alors on passe au j-ème moule
    # En restant dans l'état consommation 
    else:
        if debut < t < fin:
            print("Ici3", Mg, PFC_Consom, cadence_consommation_kg_min, quantite_moules_a_produire, quantite_moules_restants)

        liste_quantites_moules_a_produire[0] = quantite_moules_restants
        etat = "etat_Consom"
    
    return Mg, PFC_Consom, etat


# Boucle de simulation
while t < tmax :
    if etat == "etat_Consom":
        masse_grappe_moule = liste_masses_grappes_moules[0] 
        cadence_moule_par_heure = liste_cadences_moule_par_heure[0]
        quantite_moules_a_produire  = liste_quantites_moules_a_produire[0]
        
        # La cadence de consommation en kg/min du i-ème modèle
        cadence_consommation_kg_min = cadence_moule_par_heure / 60 * masse_grappe_moule  # en kg/min

        # Quantités de moules du i-ème modèle produits en une minute
        cadence_moule_par_minute = cadence_consommation_kg_min / masse_grappe_moule # en unités/min
        
        if  debut < t and  t < fin :
            print('t = ',t)
            print("Ici1", Mg, PFC_Consom, cadence_consommation_kg_min,quantite_moules_a_produire)
        
        # Si l'on peut produire des moules (c-a-d Mg et PFC sont okay) alors on on le fait !!
        if (PFC_Consom - cadence_consommation_kg_min >= PFCmin) and (Mg - eC >= Mgmin):
            
            Mg, PFC_Consom, etat = ProduirMoulesParMinute(Mg, PFC_Consom, quantite_moules_a_produire, cadence_consommation_kg_min, cadence_moule_par_minute)
                
        else:
            # Ajout poche car Mg ou PFC insuffisant !!
            PFC_Consom += poche_i
            Mg = Mgmax 


            Mg, PFC_Consom, etat = ProduirMoulesParMinute(Mg, PFC_Consom, quantite_moules_a_produire, cadence_consommation_kg_min, cadence_moule_par_minute)
            

            if numPoche < 4:
                numPoche += 1
            else :
                numPoche = 1
                numFours += 1


            if PFC_Consom > PFCmax :
                print("On a depasser pas la capacité du Fours de Coulée")     

    elif etat == "etat_Serie":
        # print("Changement de Serie")
        Mg -= eC
        temps_serie_courant -= 1
        
        # Si le Mg n'est pas bon alors on ajoute de la fonte pour la prochaine serie !! 
        if (PFC_Consom - cadence_consommation_kg_min <= PFCmin) and (Mg - eC <= Mgmin):
            PFC_Consom += poche_i
            Mg = Mgmax
            print("Mg pas bon pendant le changement de Serie") 

        if temps_serie_courant <= 0:
            if  debut < t and  t < fin :
                print("Fin Temps Serie",Mg, PFC_Consom, cadence_consommation_kg_min)
            
            temps_serie_courant = temps_serie
            etat = "etat_Consom"
        else:
            etat == "etat_Serie" 

    elif etat == "etat_Panne":
        Mg -= eC
        temps_Panne_courant -= 1
        
        if temps_Panne_courant <= 0:
            etat = "etat_Consom"
            if  debut < t and  t < fin :
                print("Fin Panne",Mg, PFC_Consom, cadence_consommation_kg_min)
            temps_Panne_courant = temps_Panne
        else:
            etat == "etat_Panne"

    elif etat == "etat_Fin":
        break
    
    # Calcul du delai_avt_traitement et DurreLimite
    DurreLimite = DurreLimite
    delai_avt_traitement = delai_avt_traitement

    

    # Mise à jour du temps
    t += 1
    cle = "Four 1"
    cle = f"Four {numFours}" 
    PFC_Consomvalues_dict[cle].append(PFC_Consom)
    Mgvalues_dict[cle].append(Mg)
    times_dict[cle].append(t)



In [94]:
import plotly.graph_objects as go

# Fonction pour insérer le dernier élément de la clé actuelle au début de la liste de la clé suivante
def inserer_dernier_element(liste_cle, times_dict, PFC_Consomvalues_dict, Mgvalues_dict):
    for i in range(len(liste_cle) - 1):
        derniere_valeur_t = times_dict[liste_cle[i]][-1] if times_dict[liste_cle[i]] else None
        derniere_valeur_PFC = PFC_Consomvalues_dict[liste_cle[i]][-1] if PFC_Consomvalues_dict[liste_cle[i]] else None
        derniere_valeur_Mg = Mgvalues_dict[liste_cle[i]][-1] if Mgvalues_dict[liste_cle[i]] else None
        if derniere_valeur_t is not None:
            times_dict[liste_cle[i + 1]] = [derniere_valeur_t] + times_dict[liste_cle[i + 1]]
            PFC_Consomvalues_dict[liste_cle[i + 1]] = [derniere_valeur_PFC] + PFC_Consomvalues_dict[liste_cle[i + 1]]
            Mgvalues_dict[liste_cle[i + 1]] = [derniere_valeur_Mg] + Mgvalues_dict[liste_cle[i + 1]]


def Plot_consom(PFC_values_dict, Mgvalues_dict, times_dict, PFCmin, PFCmax, Mgmin, Mgmax):
    # Dictionnaire de couleurs pour chaque clé
    color_dict = {
        "chargement_et_fusion": 'royalblue',
        "Four 1": 'firebrick',
        "Four 2": 'green',
        "Four 3": 'orange',
        "Four 4": 'purple',
        "Four 5": 'royalblue'
    }


    # Initialisation des dictionnaires
    cles = ["chargement_et_fusion", "Four 1", "Four 2", "Four 3", "Four 4", "Four 5"]

    # Appliquer la fonction aux trois dictionnaires
    inserer_dernier_element(cles, times_dict, PFC_Consomvalues_dict, Mgvalues_dict)


    # Create Plotly figure for PFC
    fig_pfc = go.Figure()

    # Add PFC lines for each key
    for key in PFC_values_dict.keys():
        fig_pfc.add_trace(go.Scatter(x=times_dict[key], y=PFC_values_dict[key], mode='lines', name=f'PFC over time ({key})', line=dict(color=color_dict[key])))

    # Add horizontal lines for PFCmin and PFCmax
    tmin, tmax = times_dict["Four 1"][0],times_dict["Four 5"][-1]
    fig_pfc.add_shape(type="line", x0=tmin, y0=PFCmin, x1=tmax, y1=PFCmin, line=dict(color="firebrick", width=2, dash="dash"), name="PFCmin")
    fig_pfc.add_shape(type="line", x0=tmin, y0=PFCmax, x1=tmax, y1=PFCmax, line=dict(color="green", width=2, dash="dash"), name="PFCmax")

    # Customize layout for PFC figure
    fig_pfc.update_layout(title='Evolution de la fonte dans le Four de Coulée au cours du temps (PFC)', 
                          xaxis_title='Time (minutes)', yaxis_title='PFC (kg)', 
                          template='plotly', showlegend=True,
                          annotations=[
                              dict(x=tmax, y=PFCmin, xref="x", yref="y", text="PFCmin", showarrow=True, arrowhead=2, ax=20, ay=-30),
                              dict(x=tmax, y=PFCmax, xref="x", yref="y", text="PFCmax", showarrow=True, arrowhead=2, ax=20, ay=-30)
                          ])

    # Show the PFC plot
    fig_pfc.show()

    # Create Plotly figure for Mg
    fig_mg = go.Figure()

    # Add Mg lines for each key
    for key in Mgvalues_dict.keys():
        fig_mg.add_trace(go.Scatter(x=times_dict[key], y=Mgvalues_dict[key], mode='lines', name=f'Mg over time ({key})', line=dict(color=color_dict[key])))

    # Add horizontal lines for Mgmin and Mgmax
    fig_mg.add_shape(type="line", x0=tmin, y0=Mgmin, x1=tmax, y1=Mgmin, line=dict(color="firebrick", width=2, dash="dash"), name="Mgmin")
    fig_mg.add_shape(type="line", x0=tmin, y0=Mgmax, x1=tmax, y1=Mgmax, line=dict(color="green", width=2, dash="dash"), name="Mgmax")

    # Customize layout for Mg figure
    fig_mg.update_layout(title='Evolution de la fonte dans le Four de Coulée au cours du temps (Mg)', 
                         xaxis_title='Time (minutes)', yaxis_title='Mg (kg)', 
                         template='plotly', showlegend=True,
                         annotations=[
                              dict(x=tmax, y=Mgmin, xref="x", yref="y", text="Mgmin", showarrow=True, arrowhead=2, ax=20, ay=-30),
                              dict(x=tmax, y=Mgmax, xref="x", yref="y", text="Mgmax", showarrow=True, arrowhead=2, ax=20, ay=-30)
                          ])

    # Show the Mg plot
    fig_mg.show()

# Call the function with the appropriate dictionaries and parameters
Plot_consom(PFC_Consomvalues_dict, Mgvalues_dict, times_dict, PFCmin, PFCmax, Mgmin, Mgmax)


# Autres Brouillon


Tu es un mathématicien et du dois effectuer cette tache suivant :

Faire varier la fonction Consomation_fonteCoulee au cours du temps en min
En conservant le temps t, La consomation PFC_Consom et 
le pourcentage de Mg dans un dictionnaire : times_dict, PFC_Consomvalues_dict, Mgvalues_dict
On souhaite interompre la consomation avec une variable la Panne :temps_Panne.
Pendant la Panne : le pourcentage de Mg continue à varier.
Cependant la panne peut intervenir à n'importe quel momment dans le temps et peut etre recurent, 
c'est un bouton que l'utilisateurs active ou non en donnant la durrée de la Panne temps_Panne


n'hésite pas a modifier  Consomation_fonteCoulee pour repondre à ses besoins !!!
Si tu as des questions ou s'il te manque certaines informations, n'hésite pas à me les demander !!
